In [27]:
### Import Functions
from DirectFunctions import import_fx_data, direct_daily_rets, get_pairs, compute_hedge_ratio, calculate_hedged_unhedged
import pandas as pd


In [28]:
### Call Function to Load in Rate Data
all_forex_pairs = [
    'EURUSD=X', 'GBPUSD=X', 'USDJPY=X', 'USDCHF=X', 'AUDUSD=X', 'NZDUSD=X',
    'USDCAD=X', 'EURGBP=X', 'EURJPY=X', 'EURCHF=X', 'EURAUD=X', 'EURNZD=X',
    'EURCAD=X', 'GBPAUD=X', 'GBPJPY=X', 'GBPCHF=X', 'AUDJPY=X', 'AUDCHF=X',
    'AUDNZD=X', 'NZDJPY=X', 'NZDCHF=X', 'CADJPY=X', 'CADCHF=X', 'CHFJPY=X'
]
start_date = '2021-01-01'
forex_data = import_fx_data(all_forex_pairs, start_date)
forex_data.tail()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Date,EURUSD=X,GBPUSD=X,USDJPY=X,USDCHF=X,AUDUSD=X,NZDUSD=X,USDCAD=X,EURGBP=X,EURJPY=X,...,GBPJPY=X,GBPCHF=X,AUDJPY=X,AUDCHF=X,AUDNZD=X,NZDJPY=X,NZDCHF=X,CADJPY=X,CADCHF=X,CHFJPY=X
898,2024-06-12,1.074033,1.274048,157.104996,0.89778,0.660770,0.614602,1.37549,0.84290,168.720993,...,200.143005,1.14372,103.805000,0.593226,1.075130,96.542999,0.55166,114.214996,0.65264,174.970001
899,2024-06-13,1.081315,1.279623,156.800003,0.89426,0.666290,0.618682,1.37219,0.84500,169.520004,...,200.630997,1.14400,104.468002,0.595810,1.076930,96.992996,0.55316,114.273003,0.65166,175.321335
900,2024-06-14,1.073883,1.275510,157.128006,0.89393,0.663310,0.616101,1.37410,0.84182,168.725998,...,200.419998,1.14024,104.214996,0.592500,1.076576,96.791000,0.55065,114.331001,0.65041,175.755432
901,2024-06-17,1.070217,1.268311,157.546005,0.89073,0.661288,0.613742,1.37330,0.84380,168.604004,...,199.802994,1.12963,104.129997,0.588600,1.077420,96.678001,0.54610,114.680000,0.64846,176.854004
902,2024-06-18,1.073537,1.270745,157.675995,0.88846,0.661826,0.613798,1.37195,0.84457,169.229004,...,200.362000,1.12898,104.353996,0.587990,NaN,96.744003,NaN,114.924004,0.64753,177.451996


In [29]:
### Call Function to Calculate Daily Returns
daily_returns = direct_daily_rets(forex_data)
print(daily_returns.head())


        Date  EURUSD=X  GBPUSD=X  USDJPY=X  USDCHF=X  AUDUSD=X  NZDUSD=X  \
0 2021-01-04  0.005782  0.000818  0.000126 -0.016702  0.001211  0.002282   
1 2021-01-05  0.000074 -0.008228 -0.001104 -0.002106 -0.005355 -0.002792   
2 2021-01-06  0.003973  0.004270 -0.004335 -0.003472  0.012384  0.010951   
3 2021-01-07  0.003320 -0.000313  0.003370 -0.000125  0.006567  0.006736   
4 2021-01-08 -0.005645 -0.004084  0.007435  0.008063 -0.006141 -0.005514   

   USDCAD=X  EURGBP=X  EURJPY=X  ...  GBPJPY=X  GBPCHF=X  AUDJPY=X  AUDCHF=X  \
0  0.030138  0.011583  0.000174  ...  0.000829 -0.014968  0.015680 -0.007623   
1  0.004630  0.008300 -0.000909  ... -0.009373 -0.010321 -0.006695 -0.008367   
2 -0.008317 -0.000211 -0.000491  ... -0.000029  0.000811  0.008157  0.009320   
3 -0.000734  0.003513  0.006659  ...  0.002937 -0.000460  0.009922  0.006615   
4  0.001453 -0.001568  0.001746  ...  0.003406  0.003963  0.001553  0.002319   

   AUDNZD=X  NZDJPY=X  NZDCHF=X  CADJPY=X  CADCHF=X  CHFJPY=X 

In [30]:
### Function Call
pair_combos_list = get_pairs(all_forex_pairs)
pair_combos_list


[['EURUSD=X', 'GBPUSD=X'],
 ['EURUSD=X', 'USDJPY=X'],
 ['EURUSD=X', 'USDCHF=X'],
 ['EURUSD=X', 'AUDUSD=X'],
 ['EURUSD=X', 'NZDUSD=X'],
 ['EURUSD=X', 'USDCAD=X'],
 ['EURUSD=X', 'EURGBP=X'],
 ['EURUSD=X', 'EURJPY=X'],
 ['EURUSD=X', 'EURCHF=X'],
 ['EURUSD=X', 'EURAUD=X'],
 ['EURUSD=X', 'EURNZD=X'],
 ['EURUSD=X', 'EURCAD=X'],
 ['EURUSD=X', 'GBPAUD=X'],
 ['EURUSD=X', 'GBPJPY=X'],
 ['EURUSD=X', 'GBPCHF=X'],
 ['EURUSD=X', 'AUDJPY=X'],
 ['EURUSD=X', 'AUDCHF=X'],
 ['EURUSD=X', 'AUDNZD=X'],
 ['EURUSD=X', 'NZDJPY=X'],
 ['EURUSD=X', 'NZDCHF=X'],
 ['EURUSD=X', 'CADJPY=X'],
 ['EURUSD=X', 'CADCHF=X'],
 ['EURUSD=X', 'CHFJPY=X'],
 ['GBPUSD=X', 'USDJPY=X'],
 ['GBPUSD=X', 'USDCHF=X'],
 ['GBPUSD=X', 'AUDUSD=X'],
 ['GBPUSD=X', 'NZDUSD=X'],
 ['GBPUSD=X', 'USDCAD=X'],
 ['GBPUSD=X', 'EURGBP=X'],
 ['GBPUSD=X', 'EURJPY=X'],
 ['GBPUSD=X', 'EURCHF=X'],
 ['GBPUSD=X', 'EURAUD=X'],
 ['GBPUSD=X', 'EURNZD=X'],
 ['GBPUSD=X', 'EURCAD=X'],
 ['GBPUSD=X', 'GBPAUD=X'],
 ['GBPUSD=X', 'GBPJPY=X'],
 ['GBPUSD=X', 'GBPCHF=X'],
 

In [31]:
### Append Long Position, Short Position, Hedged returns, and Unhedged Returns Columns
# Create an empty DataFrame to store hedge ratios
hedge_ratios_df = pd.DataFrame(columns = ['Pair1', 'Pair2', 'Hedge Ratio', 'Long Position', 'Short Position', 'Hedged Returns', 'Unhedged Returns'])
long_pos = 1000
# Loop over pair combinations to compute hedge ratio and returns
for pairing in pair_combos_list:
    try:
        hedge_ratio = compute_hedge_ratio(pairing, daily_returns)
        short_pos, hedged_cumulative_rets, unhedged_cumulative_rets = calculate_hedged_unhedged(long_pos, pairing, daily_returns, hedge_ratio)
        new_row = pd.DataFrame({
            'Pair1': [pairing[0]], 
            'Pair2': [pairing[1]], 
            'Hedge Ratio': [hedge_ratio], 
            'Long Position': [long_pos], 
            'Short Position': [short_pos],
            'Hedged Returns': [hedged_cumulative_rets.iloc[-1]], 
            'Unhedged Returns': [unhedged_cumulative_rets.iloc[-1]]
        })
        hedge_ratios_df = pd.concat([hedge_ratios_df, new_row], ignore_index=True)
    except Exception as e:
        print(f"Error computing hedge ratio for {pairing}: {e}")

# Display the DataFrame
print(hedge_ratios_df.tail())


        Pair1     Pair2  Hedge Ratio Long Position  Short Position  \
271  NZDCHF=X  CADCHF=X     0.465844          1000     -465.843816   
272  NZDCHF=X  CHFJPY=X    -0.249347          1000      249.347418   
273  CADJPY=X  CADCHF=X     0.406910          1000     -406.909674   
274  CADJPY=X  CHFJPY=X     0.590483          1000     -590.482676   
275  CADCHF=X  CHFJPY=X    -0.269894          1000      269.894448   

     Hedged Returns  Unhedged Returns  
271      874.645716        847.192123  
272      943.317366        847.192123  
273     1468.758380       1427.237265  
274     1117.047816       1427.237265  
275     1046.762520        932.234426  


In [32]:
### Filter for Most Profitable Hedged Pairing
# Sort the DataFrame by the 'Hedged Returns' column in descending order
most_profit_hedged = hedge_ratios_df.sort_values(by = 'Hedged Returns', ascending = False).head(1)
#print(most_profit_hedged)

### Display Results
pair1 = most_profit_hedged.iloc[0]['Pair1']
pair2 = most_profit_hedged.iloc[0]['Pair2']
hedge_ratio = most_profit_hedged.iloc[0]['Hedge Ratio']
long_position = most_profit_hedged.iloc[0]['Long Position']
short_position = most_profit_hedged.iloc[0]['Short Position']
hedged_returns = most_profit_hedged.iloc[0]['Hedged Returns']
unhedged_returns = most_profit_hedged.iloc[0]['Unhedged Returns']

print(f"Most Profitable Pairings: {pair1} and {pair2}")
print(f"Hedge Ratio: {hedge_ratio:.6f}")
print(f"Long Position: {long_position}")
print(f"Short Position: {short_position:.6f}")
print("\nReturns:")
print(f"Hedged Returns: {hedged_returns:.6f}")
print(f"Unhedged Returns: {unhedged_returns:.6f}")


Most Profitable Pairings: USDJPY=X and CADCHF=X
Hedge Ratio: 0.230840
Long Position: 1000
Short Position: -230.840303

Returns:
Hedged Returns: 1550.708393
Unhedged Returns: 1527.483367
